<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#More-patterns" data-toc-modified-id="More-patterns-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>More patterns</a></span><ul class="toc-item"><li><span><a href="#Existence-of-values" data-toc-modified-id="Existence-of-values-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Existence of values</a></span></li><li><span><a href="#Multiple-values" data-toc-modified-id="Multiple-values-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Multiple values</a></span></li><li><span><a href="#Regular-expressions" data-toc-modified-id="Regular-expressions-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Regular expressions</a></span><ul class="toc-item"><li><span><a href="#Find-all-two-letter-nouns-starting-with-an-aleph" data-toc-modified-id="Find-all-two-letter-nouns-starting-with-an-aleph-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Find all two-letter nouns starting with an aleph</a></span></li><li><span><a href="#Find-all-&quot;we-x&quot;-clauses-with-a-non-qal-verb" data-toc-modified-id="Find-all-&quot;we-x&quot;-clauses-with-a-non-qal-verb-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Find all "we-x" clauses with a non-qal verb</a></span></li><li><span><a href="#Find-all-glosses-with-a-space" data-toc-modified-id="Find-all-glosses-with-a-space-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Find all glosses with a space</a></span></li></ul></li></ul></li><li><span><a href="#Under-the-hood" data-toc-modified-id="Under-the-hood-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Under the hood</a></span></li><li><span><a href="#Excursion" data-toc-modified-id="Excursion-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Excursion</a></span></li><li><span><a href="#Back-to-business" data-toc-modified-id="Back-to-business-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Back to business</a></span></li><li><span><a href="#Find-the-gap" data-toc-modified-id="Find-the-gap-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Find the gap</a></span></li></ul></div>

<img align="right" src="images/dans-small.png"/>
<img align="right" src="images/tf-small.png"/>
<img align="right" src="images/etcbc.png"/>


# Search

*Search* in Text-Fabric is a template based way of looking for structural patterns in your dataset.

It is inspired by the idea of
[topographic query](http://books.google.nl/books?id=9ggOBRz1dO4C),
as worked out in 
[MQL](https://shebanq.ancient-data.org/shebanq/static/docs/MQL-Query-Guide.pdf)
which has been implemented in 
[Emdros](http://emdros.org).
See also [pitfalls of MQL](https://etcbc.github.io/bhsa/mql#pitfalls-of-mql)

Within Text-Fabric we have the unique possibility to combine the ease of formulating search templates for
complicated syntactical patterns with the power of programmatically processing the results.

This notebook will show you how to get up and running.

See the notebook
[searchFromMQL](searchFromMQL.ipynb)
for examples how MQL queries can be expressed in Text-Fabric search.

# Before we continue
Search is a big feature in Text-Fabric.
It is also a very recent addition.

##### Caution:
> There might be bugs.

Search is also costly.
Quite a bit of the implementation work has been dedicated to optimize performance.
But it is worth the price: search templates are powerful for a wide range of purposes.
I do not pretend, however, to have found optimal strategies for all 
possible search templates.

That being said, I think search might turn out helpful in many cases,
and I welcome your feedback.

*Dirk Roorda, 2016-12-23, updates 2017-10-10*

# Search command

Search is as simple as saying (just an example)

```python
for r in S.search(template): print(S.glean(r))
```

See all ins and outs in the
[search template reference](https://github.com/Dans-labs/text-fabric/wiki/Api#search-template-introduction).

All search related things use the
[`S` api](https://github.com/Dans-labs/text-fabric/wiki/Api#search).

In [1]:
from tf.fabric import Fabric
from tf.extra.bhsa import Bhsa

In [2]:
VERSION = '2017'
DATABASE = '~/github/etcbc'
BHSA = f'bhsa/tf/{VERSION}'
TF = Fabric(locations=[DATABASE], modules=[BHSA], silent=False )

This is Text-Fabric 3.4.0
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

115 features found and 0 ignored


Let us just *not* load any specific features.

In [3]:
api = TF.load('', silent=True)
api.makeAvailableIn(globals())

Let's add extra power:

In [4]:
B = Bhsa(api, 'search', version=VERSION)

**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="{provenance of this corpus}">BHSA</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/2017/0_home.html" title="{CORPUS} feature documentation">Feature docs</a> <a target="_blank" href="https://github.com/Dans-labs/text-fabric/wiki/Bhsa" title="BHSA API documentation">BHSA API</a> <a target="_blank" href="https://github.com/Dans-labs/text-fabric/wiki/api" title="text-fabric-api">Text-Fabric API</a>


This notebook online:
<a target="_blank" href="http://nbviewer.jupyter.org/github/etcbc/bhsa/blob/master/tutorial/search.ipynb">NBViewer</a>
<a target="_blank" href="https://github.com/etcbc/bhsa/blob/master/tutorial/search.ipynb">GitHub</a>


# Basic search command

We start with the most simple form of issuing a query.
Let's look for the word Elohim in undetermined phrases, only in Genesis 1-2.

All work involved in searching takes place under the hood.

In [5]:
query = '''
book book=Genesis
  chapter chapter=1|2
    verse
      phrase det=und
        word lex=>LHJM/
'''
for r in S.search(query): print(S.glean(r))

  Genesis 1:1 phrase[אֱלֹהִ֑ים ] אֱלֹהִ֑ים 
  Genesis 1:2 phrase[ר֣וּחַ אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:3 phrase[אֱלֹהִ֖ים ] אֱלֹהִ֖ים 
  Genesis 1:4 phrase[אֱלֹהִ֛ים ] אֱלֹהִ֛ים 
  Genesis 1:4 phrase[אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:5 phrase[אֱלֹהִ֤ים׀ ] אֱלֹהִ֤ים׀ 
  Genesis 1:6 phrase[אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:7 phrase[אֱלֹהִים֮ ] אֱלֹהִים֮ 
  Genesis 1:8 phrase[אֱלֹהִ֛ים ] אֱלֹהִ֛ים 
  Genesis 1:9 phrase[אֱלֹהִ֗ים ] אֱלֹהִ֗ים 
  Genesis 1:10 phrase[אֱלֹהִ֤ים׀ ] אֱלֹהִ֤ים׀ 
  Genesis 1:10 phrase[אֱלֹהִ֖ים ] אֱלֹהִ֖ים 
  Genesis 1:11 phrase[אֱלֹהִ֗ים ] אֱלֹהִ֗ים 
  Genesis 1:12 phrase[אֱלֹהִ֖ים ] אֱלֹהִ֖ים 
  Genesis 1:14 phrase[אֱלֹהִ֗ים ] אֱלֹהִ֗ים 
  Genesis 1:16 phrase[אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:17 phrase[אֱלֹהִ֖ים ] אֱלֹהִ֖ים 
  Genesis 1:18 phrase[אֱלֹהִ֖ים ] אֱלֹהִ֖ים 
  Genesis 1:20 phrase[אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:21 phrase[אֱלֹהִ֖ים ] אֱלֹהִ֖ים 
  Genesis 1:21 phrase[אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:22 phrase[אֱלֹהִ֖ים ] אֱלֹהִ֖ים 
  Genesis

We can do it prettier with the extra power.
We show the first 5 results.

In [6]:
results = B.search(query)
B.show(results, start=0, end=4)


##### Result 0



##### Result 1



##### Result 2



##### Result 3



##### Result 4


We can condense the results like in SHEBANQ: instead of showing all results one by one, we show all verses with all results in it highlighted. That way, we blur the distinction between the individual results, but it is easier to oversee where the results are.

In [7]:
results = B.search(query)
B.show(results, condensed=True, start=0, end=5)


##### Passage 0



##### Passage 1



##### Passage 2



##### Passage 3



##### Passage 4



##### Passage 5


Note that the passages are ordered in corpus order.

Note that the two results involving Genesis 1:4 show up in the same verse display.

Here is another example with condensing.
We look for clauses with a proper name in it.
We restrict to those in 1 Samuel.

In [8]:
query = '''
book book=Samuel_I
  clause
    word sp=nmpr
'''
results = B.search(query)
len(results)

1868

In [9]:
B.show(results, condensed=False, start=0, end=5)


##### Result 0



##### Result 1



##### Result 2



##### Result 3



##### Result 4



##### Result 5


Many results per verse. Here is the condensed form.
All proper names in a verse are highlighted.

In [10]:
B.show(results, condensed=True, start=0, end=2)


##### Passage 0



##### Passage 1



##### Passage 2


## More patterns

We can state feature conditions for the (non)-existence of values,
for the testing of multiple values and for testing regular expressions.

### Existence of values

We can ask for all words that have a qere.
Just leave out the feature values.

```
word qere
```

Conversely, we can ask for words without a qere.
Just add a `!` after the feature name.

```
word qere!
```

Let's test it.

In [11]:
query = '''
word
'''
results = list(S.search(query))
allWords = len(results)
print(f'{allWords:>6} words in total')

query = '''
word qere
'''
results = list(S.search(query))
qereWords = len(results)
print(f'{qereWords:>6} words with a qere')

query = '''
word qere!
'''
results = list(S.search(query))
plainWords = len(results)
print(f'{plainWords:>6} words without a qere')

print(f'qereWords + plainWords == allWords ? {qereWords + plainWords == allWords}')

426584 words in total
  1892 words with a qere
424692 words without a qere
qereWords + plainWords == allWords ? True


### Multiple values

The previous example shows it already:

```
  chapter=1|2
```

You may list as many values as you wish, for every feature.

### Regular expressions

An even more powerful way of specifying desired feature values is by regular expressions.
You can do this for string values features only.

Instead of specifying a feature condition like this

```
typ=WIm0
```

or

```
typ=WIm0|WImX
```

you can say

```
typ~WIm[0X]
```

Note that you do not use the `=` between feature name and value specification, 
but `~`.

The syntax and semantics of regular expressions are those as defined in the
[Python docs](https://docs.python.org/3/library/re.html#regular-expression-syntax).

Note, that if you need to enter a `\` in the regular expression, you have to double it.
Also, when you need a space in it, you have to put a `\` in front of it.

Here are a few examples.

#### Find all two-letter nouns starting with an aleph

In [12]:
query = '''
word sp=subs g_cons~^>.$
'''
results = list(S.search(query))
print(f'{len(results)} results')
for r in results[0:10]:
    print(S.glean(r))

816 results
אֵ֥ל 
אֵ֣ל 
אֵשׁ֙ 
אֵ֔שׁ 
אַ֤ף 
אֵ֖ל 
אֵ֔שׁ 
אֵֽשׁ׃ 
אַף־
אֵ֣שׁ 


#### Find all "we-x" clauses with a non-qal verb

If you look at the [clause types](https://etcbc.github.io/bhsa/features/hebrew/2017/typ.html)
you see a lot of types indicating that the clause starts with `we`:

```
Way0	Wayyiqtol-null clause
WayX	Wayyiqtol-X clause
WIm0	We-imperative-null clause
WImX	We-imperative-X clause
WQt0	We-qatal-null clause
WQtX	We-qatal-X clause
WxI0	We-x-imperative-null clause
WXIm	We-X-imperative clause
WxIX	We-x-imperative-X clause
WxQ0	We-x-qatal-null clause
WXQt	We-X-qatal clause
WxQX	We-x-qatal-X clause
WxY0	We-x-yiqtol-null clause
WXYq	We-X-yiqtol clause
WxYX	We-x-yiqtol-X clause
WYq0	We-yiqtol-null clause
WYqX	We-yiqtol-X clause
```

We are interested in the `We-x` and `We-X` clauses, so all clauses whose `typ` starts with `Wx` or `WX`.

There are quite a number of verb stems. By means of a regular expression we can pick everything except `qal`.

In the
[Python docs on regular expressions](https://docs.python.org/3/library/re.html#regular-expression-syntax)
we see that we can check for that by `^(?:!qal)`.

In [13]:
query = '''
clause typ~^W[xX]
  word sp=verb vs~^(?!qal)
'''
results = list(S.search(query))
print(f'{len(results)} results')
B.show(results, condensed=True, start=0, end=6)

3096 results



##### Passage 0



##### Passage 1



##### Passage 2



##### Passage 3



##### Passage 4



##### Passage 5



##### Passage 6


#### Find all glosses with a space

In [14]:
query = '''
lex gloss~[\ ] sp=subs
'''
results = list(S.search(query))
print(len(results))
B.show(results, start=0, end=4)

404



##### Result 0



##### Result 1



##### Result 2



##### Result 3



##### Result 4


If you search with regular expressions, then nodes without a value do not match any regular expression.

The regular expression `.*` matches everything.
Not all words have a qere.

So we expect the following qere to list all words that do have a qere.

In [15]:
query = '''
word qere~.*
'''
results = list(S.search(query))
matchWords = len(results)
print(
    f'{matchWords} results.'
    ' Compare this with qere words: '
    f'{qereWords}: {"Equal" if matchWords == qereWords else "Unequal"}')

1892 results. Compare this with qere words: 1892: Equal


## Under the hood

It might be helpful to peek under the hood, especially when exploring new searches.
We feed the query to the search API, which will *study* it.
The syntax will be checked, features loaded, the search space will be set up, narrowed down, 
and the fetching of results will be prepared, but not yet executed.

In order to make the query a bit more interesting, we lift the constraint that the results must be in Genesis 1-2.

In [16]:
query = '''
book
  chapter
    verse
      phrase det=und
        word lex=>LHJM/
'''

In [17]:
S.study(query)

  0.00s Checking search template ...
  0.00s Setting up search space for 5 objects ...
  1.14s Constraining search space with 4 relations ...
  1.16s Setting up retrieval plan ...
  1.18s Ready to deliver results from 2735 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results


Before we rush to the results, lets have a look at the *plan*.

In [18]:
S.showPlan()

  2.96s The results are connected to the original search template as follows:
 0     
 1 R0  book
 2 R1    chapter
 3 R2      verse
 4 R3        phrase det=und
 5 R4          word lex=>LHJM/
 6     


Here you see already what your results will look like.
Each result `r` is a *tuple* of nodes:
```
(R0, R1, R2, R3, R4)
```
that instantiate the objects in your template.

## Excursion
In case you are curious, you can get details about the search space as well:

In [22]:
S.showPlan(details=True)

Search with 5 objects and 4 relations
Results are instantiations of the following objects:
node  0-book                              (    39   choices)
node  1-chapter                           (   929   choices)
node  2-verse                             ( 23213   choices)
node  3-phrase                            ( 29954   choices)
node  4-word                              (  2601   choices)
Instantiations are computed along the following relations:
node                      0-book          (    39   choices)
edge  0-book          [[  1-chapter       (    11.1 choices)
edge  1-chapter       [[  2-verse         (    23.8 choices)
edge  2-verse         [[  3-phrase        (     0.9 choices)
edge  3-phrase        [[  4-word          (     0.1 choices)
  6.87s The results are connected to the original search template as follows:
 0     
 1 R0  book
 2 R1    chapter
 3 R2      verse
 4 R3        phrase det=und
 5 R4          word lex=>LHJM/
 6     


The part about the *nodes* shows you how many possible instantiations for each object in your template
has been found.
These are not results yet, because only combinations of instantiations
that satisfy all constraints are results.

The constraints come from the relations between the objects that you specified.
In this case, there is only an implicit relation: embedding `[[`. 
Later on we'll examine all basic relations.

The part about the *edges* shows you the constraints,
and in what order they will be computed when stitching results together.
In this case the order is exactly the order by which the relations appear in the template,
but that will not always be the case.
Text-Fabric spends some time and ingenuity to find out an optimal *stitch plan*.

Nevertheless, fetching results may take time. 

For some queries, it can take a large amount of time to walk through all results.
Even worse, it may happen that it takes a large amount of time before getting the *first* result.

This has to do with search strategies on the one hand,
and the very likely possibility to encounter *pathological* search patterns,
which have billions of results, mostly unintended.
For example, a simple query that asks for 5 words in the Hebrew Bible without further constraints,
will have 425,000 to the power of 5 results.
That is 10-e28 (a one with 28 zeros),
roughly the number of molecules in a few hundred liters of air.
That may not sound much, but it is 10,000 times the amount of bytes
that can be currently stored on the whole Internet.

Text-Fabric search is not yet done with finding optimal search strategies,
and I hope to refine its arsenal of methods in the future, depending on what you report.

## Back to business
It is always a good idea to get a feel for the amount of results, before you dive into them head-on.

In [23]:
S.count(progress=1, limit=5)

  0.00s Counting results per 1 up to 5 ...
   |     0.01s 1
   |     0.01s 2
   |     0.01s 3
   |     0.02s 4
   |     0.02s 5
  0.02s Done: 5 results


We asked for 5 results in total, with a progress message for every one.
That was a bit conservative.

In [24]:
S.count(progress=100, limit=500)

  0.00s Counting results per 100 up to 500 ...
   |     0.03s 100
   |     0.09s 200
   |     0.17s 300
   |     0.23s 400
   |     0.31s 500
  0.31s Done: 500 results


Still pretty quick, now we want to count all results.

In [25]:
S.count(progress=200, limit=-1)

  0.00s Counting results per 200 up to  the end of the results ...
   |     0.08s 200
   |     0.21s 400
   |     0.29s 600
   |     0.38s 800
  0.39s Done: 818 results


It is time to see something of those results.

In [26]:
S.fetch(limit=10)

((426585, 426624, 1414190, 651505, 4),
 (426585, 426624, 1414191, 651515, 26),
 (426585, 426624, 1414192, 651520, 34),
 (426585, 426624, 1414193, 651528, 42),
 (426585, 426624, 1414193, 651534, 50),
 (426585, 426624, 1414194, 651538, 60),
 (426585, 426624, 1414195, 651554, 81),
 (426585, 426624, 1414196, 651564, 97),
 (426585, 426624, 1414197, 651578, 127),
 (426585, 426624, 1414198, 651590, 142))

Not very informative.
Just a quick observation: look at the last column.
These are the result nodes for the `word` part in the query, indicated as `R7` by `showPlan()` before.
And indeed, they are all below 425,000, the number of words in the Hebrew Bible.

Nevertheless, we want to glean a bit more information off them.

In [27]:
for r in S.fetch(limit=10):
    print(S.glean(r))

  Genesis 1:1 phrase[אֱלֹהִ֑ים ] אֱלֹהִ֑ים 
  Genesis 1:2 phrase[ר֣וּחַ אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:3 phrase[אֱלֹהִ֖ים ] אֱלֹהִ֖ים 
  Genesis 1:4 phrase[אֱלֹהִ֛ים ] אֱלֹהִ֛ים 
  Genesis 1:4 phrase[אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:5 phrase[אֱלֹהִ֤ים׀ ] אֱלֹהִ֤ים׀ 
  Genesis 1:6 phrase[אֱלֹהִ֔ים ] אֱלֹהִ֔ים 
  Genesis 1:7 phrase[אֱלֹהִים֮ ] אֱלֹהִים֮ 
  Genesis 1:8 phrase[אֱלֹהִ֛ים ] אֱלֹהִ֛ים 
  Genesis 1:9 phrase[אֱלֹהִ֗ים ] אֱלֹהִ֗ים 


##### Caution
> It is not possible to do `len(S.fetch())`.
Because `fetch()` is a *generator*, not a list.
It will deliver a result every time it is being asked and for as long as there are results,
but it does not know in advance how many there will be.

>Fetching a result can be costly, because due to the constraints, a lot of possibilities
may have to be tried and rejected before a the next result is found.

> That is why you often see results coming in at varying speeds when counting them.

This search template has some pretty tight constraints on one of its objects,
so the amount of data to deal with is pretty limited.

Let us turn to a template where this is not so.

In [28]:
query = '''
# test
# verse book=Genesis chapter=2 verse=25
verse
  clause
                                 
    p1:phrase
        w1:word
        w3:word
        w1 < w3

    p2:phrase
        w2:word
        w1 < w2 
        w3 > w2
    
    p1 < p2   
'''

A couple of remarks.

* some objects have got a name
* there are additional relations specified between named objects
* `<` means: *comes before*, and `>`: *comes after* in the canonical order for nodes,
  which for words means: comes textually before/after, but for other nodes the meaning
  is explained [here](https://github.com/Dans-labs/text-fabric/wiki/Api#sorting-nodes)
* later on we describe those relations in more detail

##### Note on order
> Look at the words `w1` and `w3` below phrase `p1`.
Although in the template `w1` comes before `w3`, this is not 
translated in a search constraint of the same nature.

> Order between objects in a template is never significant, only embedding is.

Because order is not significant, you have to specify order yourself, using relations.

It turns out that this is better than the other way around.
In MQL order *is* significant, and it is very difficult to 
search for `w1` and `w2` in any order.
Especially if your are looking for more than 2 complex objects with lots of feature
conditions, your search template would explode if you had to spell out all
possible permutations. See the example of Reinoud Oosting below.

##### Note on gaps
> Look at the phrases `p1` and `p2`.
We do not specify an order here, only that they are different.
In order to prevent duplicated searches with `p1` and `p2` interchanged, we even 
stipulate that `p1 < p2`.
There are many spatial relationships possible between different objects.
In many cases, neither the one comes before the other, nor vice versa.
They can overlap, one can occur in a gap of the other, they can be completely disjoint
and interleaved, etc.

In [29]:
S.study(query)

  0.00s Checking search template ...
  0.00s Setting up search space for 7 objects ...
  0.51s Constraining search space with 10 relations ...
  0.54s Setting up retrieval plan ...
  0.59s Ready to deliver results from 1897440 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results


That was quick!
Well, Text-Fabric knows that narrowing down the search space in this case would take ages,
without resulting in a significantly shrunken space.
So it skips doing so for most constraints.

Let us see the plan, with details.

In [30]:
S.showPlan(details=True)

Search with 7 objects and 10 relations
Results are instantiations of the following objects:
node  0-verse                             ( 23213   choices)
node  1-clause                            ( 88101   choices)
node  2-phrase                            (253187   choices)
node  3-word                              (426584   choices)
node  4-word                              (426584   choices)
node  5-phrase                            (253187   choices)
node  6-word                              (426584   choices)
Instantiations are computed along the following relations:
node                      0-verse         ( 23213   choices)
edge  0-verse         [[  1-clause        (     3.0 choices)
edge  1-clause        [[  2-phrase        (     2.3 choices)
edge  2-phrase        [[  3-word          (     1.5 choices)
edge  2-phrase        [[  4-word          (     1.7 choices)
edge  3-word          <   4-word          (213292.0 choices)
edge  1-clause        [[  5-phrase        (     2.9 choi

As you see, we have a hefty search space here.
Let us play with the `count()` function.

In [31]:
S.count(progress=10, limit=100)

  0.00s Counting results per 10 up to 100 ...
   |     0.16s 10
   |     0.16s 20
   |     0.16s 30
   |     0.18s 40
   |     0.19s 50
   |     0.19s 60
   |     0.22s 70
   |     0.22s 80
   |     0.22s 90
   |     0.22s 100
  0.23s Done: 100 results


We can be bolder than this!

In [32]:
S.count(progress=100, limit=1000)

  0.00s Counting results per 100 up to 1000 ...
   |     0.19s 100
   |     0.23s 200
   |     0.23s 300
   |     0.39s 400
   |     0.44s 500
   |     0.45s 600
   |     0.51s 700
   |     0.64s 800
   |     0.66s 900
   |     0.84s 1000
  0.85s Done: 1000 results


OK, not too bad, but note that it takes a big fraction of a second to get just 100 results.

Now let us go for all of them by the thousand.

In [33]:
S.count(progress=1000, limit=-1)

  0.00s Counting results per 1000 up to  the end of the results ...
   |     0.82s 1000
   |     1.61s 2000
   |     2.22s 3000
   |     2.80s 4000
   |     3.30s 5000
   |     4.31s 6000
   |     6.43s 7000
  7.93s Done: 7618 results


See? This is substantial work.

In [34]:
for r in S.fetch(limit=10):
    print(S.glean(r))

Genesis 2:25 clause[וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽ...] phrase[שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו ] שְׁנֵיהֶם֙  הָֽ phrase[עֲרוּמִּ֔ים ] עֲרוּמִּ֔ים 
Genesis 2:25 clause[וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽ...] phrase[שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו ] שְׁנֵיהֶם֙  אָדָ֖ם  phrase[עֲרוּמִּ֔ים ] עֲרוּמִּ֔ים 
Genesis 2:25 clause[וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽ...] phrase[שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו ] שְׁנֵיהֶם֙  וְ phrase[עֲרוּמִּ֔ים ] עֲרוּמִּ֔ים 
Genesis 2:25 clause[וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽ...] phrase[שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו ] שְׁנֵיהֶם֙  אִשְׁתֹּ֑ו  phrase[עֲרוּמִּ֔ים ] עֲרוּמִּ֔ים 
Genesis 4:4 clause[וְהֶ֨בֶל הֵבִ֥יא גַם־ה֛וּא ...] phrase[הֶ֨בֶל גַם־ה֛וּא ] הֶ֨בֶל  גַם־ phrase[הֵבִ֥יא ] הֵבִ֥יא 
Genesis 4:4 clause[וְהֶ֨בֶל הֵבִ֥יא גַם־ה֛וּא ...] phrase[הֶ֨בֶל גַם־ה֛וּא ] הֶ֨בֶל  ה֛וּא  phrase[הֵבִ֥יא ] הֵבִ֥יא 
Genesis 10:21 clause[גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־...] phrase[גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַ...] גַּם־ אֲחִ֖י  phrase[אֲבִי֙ כָּל

As a check, here is some code that looks for basically the same phenomenon:
a phrase within the gap of another phrase.
It does not use search, and it gets a bit more focused results, in half the time compared
to the search with the template.

##### Hint
> If you are comfortable with programming, and what you look for is fairly generic,
you may be better off without search, provided you can translate your insight in the
data into an effective procedure within Text-Fabric.
But wait till we are completely done with this example!

In [19]:
indent(reset=True)
info('Getting gapped phrases')
results = []
for v in F.otype.s('verse'):
    for c in L.d(v, otype='clause'):
        ps = L.d(c, otype='phrase')
        first = {}
        last = {}
        slots = {}
        # make index of phrase boundaries
        for p in ps:
            words = L.d(p, otype='word')
            first[p] = words[0]
            last[p] = words[-1]
            slots[p] = set(words)
        for p1 in ps:
            for p2 in ps:
                if p2 < p1: continue
                if len(slots[p1] & slots[p2]) != 0: continue
                if first[p1] < first[p2] and last[p2] < last[p1]:
                    results.append((v, c, p1, p2, first[p1], first[p2], last[p2], last[p1]))
info('{} results'.format(len(results)))
for r in results[0:10]:
    print(r)

  0.00s Getting gapped phrases
  3.13s 368 results
(1414245, 427767, 652147, 652148, 1159, 1160, 1160, 1164)
(1414273, 427889, 652504, 652505, 1720, 1721, 1721, 1723)
(1414445, 428386, 654102, 654103, 4819, 4821, 4824, 4828)
(1414505, 428569, 654678, 654679, 5803, 5805, 5806, 5809)
(1414509, 428585, 654725, 654726, 5868, 5869, 5870, 5875)
(1414542, 428692, 655061, 655062, 6515, 6521, 6521, 6530)
(1414594, 428886, 655642, 655643, 7431, 7432, 7433, 7437)
(1414651, 429128, 656353, 656354, 8502, 8507, 8507, 8520)
(1414651, 429128, 656353, 656355, 8502, 8508, 8510, 8520)
(1414740, 429497, 657505, 657506, 10284, 10287, 10287, 10291)


But we can use the pretty printing of `S.glean()` and `B.show()` here as well, even though we have
not used search!

In [20]:
for r in results[0:10]: print(S.glean(r))

Genesis 2:25 clause[וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽ...] phrase[שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו ] phrase[עֲרוּמִּ֔ים ] שְׁנֵיהֶם֙  עֲרוּמִּ֔ים  עֲרוּמִּ֔ים  אִשְׁתֹּ֑ו 
Genesis 4:4 clause[וְהֶ֨בֶל הֵבִ֥יא גַם־ה֛וּא ...] phrase[הֶ֨בֶל גַם־ה֛וּא ] phrase[הֵבִ֥יא ] הֶ֨בֶל  הֵבִ֥יא  הֵבִ֥יא  ה֛וּא 
Genesis 10:21 clause[גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־...] phrase[גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַ...] phrase[אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר ] גַּם־ אֲבִי֙  עֵ֔בֶר  גָּדֹֽול׃ 
Genesis 12:17 clause[וַיְנַגַּ֨ע יְהוָ֧ה׀ אֶת־פַּרְעֹ֛ה ...] phrase[אֶת־פַּרְעֹ֛ה וְאֶת־בֵּיתֹ֑ו ] phrase[נְגָעִ֥ים גְּדֹלִ֖ים ] אֶת־ נְגָעִ֥ים  גְּדֹלִ֖ים  בֵּיתֹ֑ו 
Genesis 13:1 clause[וַיַּעַל֩ אַבְרָ֨ם מִמִּצְרַ֜יִם ...] phrase[אַבְרָ֨ם ה֠וּא וְאִשְׁתֹּ֧ו וְ...] phrase[מִמִּצְרַ֜יִם ] אַבְרָ֨ם  מִ מִּצְרַ֜יִם  כָל־
Genesis 14:16 clause[וְגַם֩ אֶת־לֹ֨וט אָחִ֤יו ...] phrase[גַם֩ אֶת־לֹ֨וט אָחִ֤יו וּ...] phrase[הֵשִׁ֔יב ] גַם֩  הֵשִׁ֔יב  הֵשִׁ֔יב  עָֽם׃ 
Genesis 17:7 clause[לִהְיֹ֤ות לְךָ֙ לֵֽאלֹהִ֔ים ...] phrase[לְךָ֙ וּֽלְז

In [21]:
B.show(results, start=0, end=9)


##### Result 0



##### Result 1



##### Result 2



##### Result 3



##### Result 4



##### Result 5



##### Result 6



##### Result 7



##### Result 8



##### Result 9


Further on we have another example with gaps, and we get the results by means of a search template in a slightly other way.

# Refine the search template

A second look at the results of our search template reveals 
that there are multiple results per pair of phrases,
because there are in general multiple words in both
phrases that satisfy the condition.
We can make the search template stricter,
by requiring alignment of the words with the starts and ends of the phrases
they are in.

For this, we employ a convenient device in search templates that we have not explained yet.

**Before each atom we may put a relational operator.**

The meaning is that this relation holds between the preceding atom and the current one.
If there is a lonely operator all by itself on a line, it means that 
this relation holds between the preceding sibling and the parent.

These operators are very handy to indicate that there is an order between siblings,
and also that a child should start or end where the parent starts or ends.

In [22]:
query = '''
verse
  clause
                                 
    p1:phrase
        =: w1:word
        <  w3:word
        :=

    p2:phrase
        =: w2:word
    
    p1 < p2
    w1 < p2
    w2 < w3
    
'''

The line 

```
=: w1:word
``` 

constrains word `w1` to start exactly at the start of its parent, phrase `p1`.

The line

```
<  w3:word
```

constrains the preceding sibling `w1` to come before `w3` in the canonical node ordering.
Because `w1` and `w3` are words, this means that `w1` comes textually before `w3`.

The line 

```
:=
``` 

constrains the preceding sibling, word `w3` to end exactly at the end of its parent,
phrase `p1`.

The line 

```
=: w2:word
```

constrains word `w2` to start exactly at the start of its parent, phrase `p2`.

Given two phrases `p1` and `p2`, the positions of all three words `w1`, `w2`, `w3` are fixed, so for every pair `p1`, `p2` that satisfies the conditions, 
there is exactly one result.

In [39]:
S.study(query)
S.showPlan(details=True)
S.count(progress=100, limit=-1)
for r in S.fetch(limit=10): print(S.glean(r))

  0.00s Checking search template ...
  0.05s Setting up search space for 7 objects ...
  0.54s Constraining search space with 13 relations ...
  0.59s Setting up retrieval plan ...
  0.67s Ready to deliver results from 1897440 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
Search with 7 objects and 13 relations
Results are instantiations of the following objects:
node  0-verse                             ( 23213   choices)
node  1-clause                            ( 88101   choices)
node  2-phrase                            (253187   choices)
node  3-word                              (426584   choices)
node  4-word                              (426584   choices)
node  5-phrase                            (253187   choices)
node  6-word                              (426584   choices)
Instantiations are computed along the following relations:
node                      0-verse         ( 23213   choices)
edge  0-verse         [[  1-clause        ( 

And here we have exactly the same results as our hand-written piece of code.

> Note
Now, with the "duplicate" results prevented, the search with the template has
only a slight performance overhead compared to the manual piece of code!

But beware of complications. 
Search templates are powerful, but sometimes they lead to a different
result set from what you might think.
Here is an example.

# A tricky example

Suppose we want to count the clauses consisting of exactly two phrases.
The following template should do it:
a clause, starting with a phrase, followed by an adjacent phrase,
which terminates the clause.

In [40]:
query = '''
clause
    =: phrase
    <: phrase
    :=
'''

In [41]:
S.study(query)
S.showPlan(details=True)
qresults = sorted(r[0] for r in S.fetch())
info(f'Done: found {len(qresults)}')

  0.00s Checking search template ...
  0.00s Setting up search space for 3 objects ...
  0.18s Constraining search space with 5 relations ...
  0.20s Setting up retrieval plan ...
  0.24s Ready to deliver results from 594475 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
Search with 3 objects and 5 relations
Results are instantiations of the following objects:
node  0-clause                            ( 88101   choices)
node  1-phrase                            (253187   choices)
node  2-phrase                            (253187   choices)
Instantiations are computed along the following relations:
node                      0-clause        ( 88101   choices)
edge  0-clause        :=  2-phrase        (     1.0 choices)
edge  2-phrase        ]]  0-clause        (     1.0 choices)
edge  2-phrase        :>  1-phrase        (     1.0 choices)
edge  1-phrase        =:  0-clause        (     0.2 choices)
edge  1-phrase        ]]  0-clause        (    

Let us check this with a piece of hand-written code.

In [42]:
indent(reset=True)
info('counting ...')

cresults = []
for c in F.otype.s('clause'):
    wc = L.d(c, otype='word')
    ps = L.d(c, otype='phrase')
    if len(ps) == 2:
        (fp, lp) = ps
        wf = L.d(fp, otype='word')
        wl = L.d(lp, otype='word')
        if wf[0] == wc[0] and wf[-1] + 1 == wl[0] and wl[-1] == wc[-1]:
            cresults.append(c)
cresults = sorted(cresults)
info(f'Done: found {len(cresults)}')

  0.00s counting ...
  1.40s Done: found 23399


Strange, we end up with less cases. What is happening? Let us compare the results.
We look at the first result where both methods diverge.

In [43]:
diff = [x for x in zip(qresults, cresults) if x[0] != x[1]]
print(f'{len(diff)} differences')
print(diff[0])

23119 differences
(428692, 428697)


Let's look at the phrases of the first difference:

In [44]:
for p in L.d(diff[0][0], otype='phrase'):
    print(f'Phrase {p} has words {L.d(p, otype="word")}')

Phrase 655060 has words [6514]
Phrase 655061 has words [6515, 6516, 6517, 6518, 6519, 6520, 6522, 6523, 6524, 6525, 6526, 6527, 6528, 6529, 6530]
Phrase 655062 has words [6521]


This clause has three phrases, but the third one lies inside the second one,
so that the clause indeed satisfies the pattern of two adjacent phrases.

Can we adjust the pattern to exclude cases like this? 
At the moment, our search template mechanism is not powerful enough for that.

We can count how often it happens, however. 
We require a third phrase to be present, not equal to one of the first two ones.

In [45]:
query = '''
clause
    =: p1:phrase
    <: p2:phrase
    :=
    p3:phrase
    p1 # p3
    p2 # p3
'''

In [46]:
S.study(query)
S.showPlan()
rresults = sorted(r[0] for r in S.fetch())
info(f'Done: found {len(rresults)}')

  0.00s Checking search template ...
  0.02s Setting up search space for 4 objects ...
  0.26s Constraining search space with 8 relations ...
  0.29s Setting up retrieval plan ...
  0.34s Ready to deliver results from 847662 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.34s The results are connected to the original search template as follows:
 0     
 1 R0  clause
 2 R1      =: p1:phrase
 3 R2      <: p2:phrase
 4         :=
 5 R3      p3:phrase
 6         p1 # p3
 7         p2 # p3
 8     
  1.34s Done: found 118


But we have to filter this, because per `p1`, `p2` there might be multiple `p3` that satisfy the constraints.
So lets gather the set of `p1`, `p2` pairs.

In [47]:
len(set(rresults))

84

And this is exactly the difference between 
the number of results of the search template and the hand-written piece of code.

# Testing basic relations

Basic relations are about the identity spatial ordering of objects.
Are they the same, do they occupy the same slots, do they overlap, is one embedded in the other,
does one come before the other?

We also have edge features, that specify relationships between nodes.

Although the basic relationships are easy to define, and even easy to implement,
they may be very costly to use. 
When searching, most of them have to be computed very many times.

Some of them have been precomputed and stored in an index, e.g. the embedding relationships.
They can be used without penalty.

Other relations are not suitable for pre-computing: most inequality relations are of that kind.
It would require an enormous amount of storage to pre-compute for each node the set of nodes that
occupy different slots. This type of relation will not be used in narrowing down the search space,
which means that it may take more time to get the results.

We are going to test all of our basic relationships here.

Let us first see what relations we have:

In [48]:
print(S.relationLegend)

                      = left equal to right (as node)
                      # left unequal to right (as node)
                      < left before right (in canonical node ordering)
                      > left after right (in canonical node ordering)
                     == left occupies same slots as right
                     && left has overlapping slots with right
                     ## left and right do not have the same slot set
                     || left and right do not have common slots
                     [[ left embeds right
                     ]] left embedded in right
                     << left completely before right
                     >> left completely after right
                     =: left and right start at the same slot
                     := left and right end at the same slot
                     :: left and right start and end at the same slot
                     <: left immediately before right
                     :> left immediately after right
   

# = (equal as node)

The `=` means that both parts are the same node. Left and right are not two things with similar properties,
no, they are one and the same thing.

Useful if the thing you search for it part of two wildly different patterns.

In [49]:
query = '''
v1:verse
  sentence
    clause rela=Objc
      phrase
        word sp=verb gn=f nu=pl
v2:verse
  sentence
    c1:clause
    c2:clause
    c3:clause
    c1 < c2
    c2 < c3
v1 = v2
'''
for r in S.search(query, limit=10): print(S.glean(r))

Ezekiel 36:36 sentence[וְיָדְע֣וּ הַגֹּויִ֗ם אֲשֶׁ֣ר ...] clause[הַנֶּ֣הֱרָסֹ֔ות ] phrase[נֶּ֣הֱרָסֹ֔ות ] נֶּ֣הֱרָסֹ֔ות  Ezekiel 36:36 sentence[וְיָדְע֣וּ הַגֹּויִ֗ם אֲשֶׁ֣ר ...] clause[וְיָדְע֣וּ הַגֹּויִ֗ם ] clause[אֲשֶׁ֣ר יִֽשָּׁאֲרוּ֮ סְבִיבֹותֵיכֶם֒ ] clause[כִּ֣י׀ אֲנִ֣י יְהוָ֗ה בָּנִ֨יתִי֙ ]
Ezekiel 36:36 sentence[וְיָדְע֣וּ הַגֹּויִ֗ם אֲשֶׁ֣ר ...] clause[הַנֶּ֣הֱרָסֹ֔ות ] phrase[נֶּ֣הֱרָסֹ֔ות ] נֶּ֣הֱרָסֹ֔ות  Ezekiel 36:36 sentence[וְיָדְע֣וּ הַגֹּויִ֗ם אֲשֶׁ֣ר ...] clause[וְיָדְע֣וּ הַגֹּויִ֗ם ] clause[אֲשֶׁ֣ר יִֽשָּׁאֲרוּ֮ סְבִיבֹותֵיכֶם֒ ] clause[הַנֶּ֣הֱרָסֹ֔ות ]
Ezekiel 36:36 sentence[וְיָדְע֣וּ הַגֹּויִ֗ם אֲשֶׁ֣ר ...] clause[הַנֶּ֣הֱרָסֹ֔ות ] phrase[נֶּ֣הֱרָסֹ֔ות ] נֶּ֣הֱרָסֹ֔ות  Ezekiel 36:36 sentence[וְיָדְע֣וּ הַגֹּויִ֗ם אֲשֶׁ֣ר ...] clause[וְיָדְע֣וּ הַגֹּויִ֗ם ] clause[אֲשֶׁ֣ר יִֽשָּׁאֲרוּ֮ סְבִיבֹותֵיכֶם֒ ] clause[נָטַ֖עְתִּי ]
Ezekiel 36:36 sentence[וְיָדְע֣וּ הַגֹּויִ֗ם אֲשֶׁ֣ר ...] clause[הַנֶּ֣הֱרָסֹ֔ות ] phrase[נֶּ֣הֱרָסֹ֔ות ] נֶּ֣הֱרָסֹ֔ות  Ezekiel 36:36 sente

# # (unequal as node)

`n # m` if `n` and `m` are not the same node.

If you write a template, and you know that one node should come before another one,
consider using `<` or `>`, which will constrain the results better.

We have seen this in action in the search for gapped phrases.

# < and > (canonical)

`n < m` if `n` comes before `m` in the
[canonical ordering](https://github.com/Dans-labs/text-fabric/wiki/Api#sorting-nodes)
of nodes.

We have seen them in action before.

# == (same slots)

Two objects are extensionally equal if they occupy exactly the same slots.

Quite an expensive relation, as you will see: nearly 30 seconds for ca. 3600 results.

In [50]:
query = '''
v:verse
    s:sentence
v == s
'''
for r in S.search(query, limit=10): print(S.glean(r))
S.count(progress=1000, limit=10000)

Isaiah 52:14 sentence[כַּאֲשֶׁ֨ר שָׁמְמ֤וּ עָלֶ֨יךָ֙ רַבִּ֔ים ...]
Proverbs 23:34 sentence[וְ֭הָיִיתָ כְּשֹׁכֵ֣ב בְּ...]
Proverbs 24:4 sentence[וּ֭בְדַעַת חֲדָרִ֣ים יִמָּלְא֑וּ ...]
Leviticus 12:1 sentence[וַיְדַבֵּ֥ר יְהוָ֖ה אֶל־מֹשֶׁ֥ה ...]
Leviticus 12:3 sentence[וּבַיֹּ֖ום הַ...]
Proverbs 24:8 sentence[מְחַשֵּׁ֥ב לְהָרֵ֑עַ לֹ֝֗ו בַּֽעַל־...]
Leviticus 12:6 sentence[וּבִמְלֹ֣את׀ יְמֵ֣י טָהֳרָ֗הּ ...]
Leviticus 13:1 sentence[וַיְדַבֵּ֣ר יְהוָ֔ה אֶל־מֹשֶׁ֥ה ...]
Isaiah 54:12 sentence[וְשַׂמְתִּ֤י כַּֽדְכֹד֙ שִׁמְשֹׁתַ֔יִךְ וּ...]
Leviticus 13:9 sentence[נֶ֣גַע צָרַ֔עַת כִּ֥י תִהְיֶ֖ה בְּ...]
  0.00s Counting results per 1000 up to 10000 ...
   |     6.62s 1000
   |       13s 2000
   |       20s 3000
    24s Done: 3601 results


# && (overlap)

Two objects overlap if and only if they share at least one slot.
This is quite costly to use in some cases.

In [51]:
query = '''
verse
    phrase
      s1:subphrase
      s2:subphrase
      s1 # s2
      s1 && s2
'''
for r in S.search(query, limit=10): print(S.glean(r))

Genesis 1:14 phrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ...] subphrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ] subphrase[לְאֹתֹת֙ ]
Genesis 1:14 phrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ...] subphrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ] subphrase[לְמֹ֣ועֲדִ֔ים ]
Genesis 1:14 phrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ...] subphrase[לְאֹתֹת֙ ] subphrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ]
Genesis 1:14 phrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ...] subphrase[לְמֹ֣ועֲדִ֔ים ] subphrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ]
Genesis 1:14 phrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ...] subphrase[לְיָמִ֖ים וְשָׁנִֽים׃ ] subphrase[יָמִ֖ים ]
Genesis 1:14 phrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ...] subphrase[לְיָמִ֖ים וְשָׁנִֽים׃ ] subphrase[שָׁנִֽים׃ ]
Genesis 1:14 phrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ...] subphrase[יָמִ֖ים ] subphrase[לְיָמִ֖ים וְשָׁנִֽים׃ ]
Genesis 1:14 phrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ...] subphrase[שָׁנִֽים׃ ] subphrase[לְיָמִ֖ים וְשָׁנִֽים׃ ]
Genesis 1:16 phrase[אֶת־שְׁנֵ֥י הַמְּאֹרֹ֖ת הַ...] subphrase[הַמְּאֹרֹ֖ת הַגְּדֹלִ֑ים ] subphrase[הַגְּדֹלִ֑ים ]
Genesis 1:16 phrase[אֶת־ש

# ## (not the same slots)

True when the two objects in question do not occupy exactly the same set of slots.
This is a very loose relationship.

It is implemented, but not yet tested, and at the moment I have not a clear use case for it.

# || (disjoint slots)

True when the two objects in question do not share any slots.
This is a rather loose relationship.

This cab be used for locating gaps: a textual object that lies inside a gap of another object.

# [[ and ]] (embedding)

`n [[ m` if object `n` embeds `m`.

`n ]] m` if object `n` lies embedded in `n`.

These relations are used implicitly in templates when there is indentation:

```
s:sentence
  p:phrase
    w1:word gn=f
    w2:word gn=m
```

The template above implicitly states the following embeddings:

* `s ]] p`
* `p ]] w1`
* `p ]] w2`

We have seen these relations in action.

# << and >> (before and after with slots)

These relations test whether one object comes before or after an other,
in the sense that the slots
occupied by the one object lie completely 
before or after the slots occupied by the other object.

In [52]:
query = '''
verse
  sentence
    c1:clause
    p:phrase
    c2:clause
    c1 << p
    c2 >> p
'''
for r in S.search(query, limit=10): print(S.glean(r))

Genesis 1:11 sentence[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] clause[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] phrase[עֹ֤שֶׂה ] clause[אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו ]
Genesis 1:11 sentence[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] clause[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] phrase[פְּרִי֙ ] clause[אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו ]
Genesis 1:11 sentence[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] clause[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] phrase[לְמִינֹ֔ו ] clause[אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו ]
Genesis 1:11 sentence[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] clause[מַזְרִ֣יעַ זֶ֔רַע ] phrase[עֹ֤שֶׂה ] clause[אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו ]
Genesis 1:11 sentence[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] clause[מַזְרִ֣יעַ זֶ֔רַע ] phrase[פְּרִי֙ ] clause[אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו ]
Genesis 1:11 sentence[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] clause[מַזְרִ֣יעַ זֶ֔רַע ] phrase[לְמִינֹ֔ו ] clause[אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו ]
Genesis 1:12 sentence[וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא ...] clause[וַתֹּוצֵ֨א הָאָ֜רֶץ ד

# =: (start at same slots)
This relation holds when the left and right hand sides are nodes that have the same first slot.
It serves to enforce the the children of a parent are textually the first things inside that
parent. We have seen it in action before.

# := (end at same slots)
This relation holds when the left and right hand sides are nodes that have the same last slot
It serves to enforce the the children of a parent are textually the last things inside that
parent. We have seen it in action before.

# :: (same start and end slots)
This relation holds when `=:` and `:=` both hold between the left and right hand sides.
It serves to look for parents with single children, or at least, where the parent is textually spanned by a single child.

In [45]:
query = '''
verse
    clause
        :: phrase
'''
for r in S.search(query, limit=10): print(S.glean(r))
S.count(progress=1000, limit=-1)

2_Samuel 8:13 clause[בְּשֻׁבֹ֕ו ] phrase[בְּשֻׁבֹ֕ו ]
Psalms 31:24 clause[כָּֽל־חֲסִ֫ידָ֥יו ] phrase[כָּֽל־חֲסִ֫ידָ֥יו ]
Psalms 31:25 clause[חִ֭זְקוּ ] phrase[חִ֭זְקוּ ]
Psalms 31:25 clause[כָּל־] phrase[כָּל־]
Psalms 32:1 clause[לְדָוִ֗ד ] phrase[לְדָוִ֗ד ]
Psalms 32:1 clause[מַ֫שְׂכִּ֥יל ] phrase[מַ֫שְׂכִּ֥יל ]
2_Samuel 9:1 clause[הֲ] phrase[הֲ]
Psalms 32:1 clause[אַשְׁרֵ֥י נְֽשׂוּי־פֶּ֗שַׁע כְּס֣וּי חֲטָאָֽה׃ ] phrase[אַשְׁרֵ֥י נְֽשׂוּי־פֶּ֗שַׁע כְּס֣וּי חֲטָאָֽה׃ ]
Psalms 32:2 clause[אַ֥שְֽׁרֵי אָדָ֗ם ] phrase[אַ֥שְֽׁרֵי אָדָ֗ם ]
Psalms 32:4 clause[סֶֽלָה׃ ] phrase[סֶֽלָה׃ ]
  0.00s Counting results per 1000 up to  the end of the results ...
   |     0.01s 1000
   |     0.03s 2000
   |     0.04s 3000
   |     0.05s 4000
   |     0.07s 5000
   |     0.09s 6000
   |     0.10s 7000
   |     0.12s 8000
   |     0.14s 9000
  0.14s Done: 9451 results


Like before, there might be extra phrases in such clauses, lying embedded in the clause-spanning phrase.

In [53]:
query = '''
verse
    clause
        :: p1:phrase
        p2:phrase
        p1 # p2
'''
for r in S.search(query, limit=10): print(S.glean(r))
S.count(progress=1000, limit=-1)


2_Samuel 22:31 clause[מָגֵ֣ן ה֔וּא לְכֹ֖ל ] phrase[מָגֵ֣ן לְכֹ֖ל ] phrase[ה֔וּא ]
Genesis 10:21 clause[גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־...] phrase[גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַ...] phrase[אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר ]
1_Kings 2:26 clause[עֲנָתֹת֙ לֵ֣ךְ עַל־שָׂדֶ֔יךָ ] phrase[עֲנָתֹת֙ עַל־שָׂדֶ֔יךָ ] phrase[לֵ֣ךְ ]
Genesis 24:24 clause[בַּת־בְּתוּאֵ֖ל אָנֹ֑כִי בֶּן־מִלְכָּ֕ה ] phrase[בַּת־בְּתוּאֵ֖ל בֶּן־מִלְכָּ֕ה ] phrase[אָנֹ֑כִי ]
Genesis 31:16 clause[לָ֥נוּ ה֖וּא וּלְבָנֵ֑ינוּ ] phrase[לָ֥נוּ וּלְבָנֵ֑ינוּ ] phrase[ה֖וּא ]
Genesis 31:53 clause[אֱלֹהֵ֨י אַבְרָהָ֜ם וֵֽאלֹהֵ֤י נָחֹור֙ ...] phrase[אֱלֹהֵ֨י אַבְרָהָ֜ם וֵֽאלֹהֵ֤י נָחֹור֙ ...] phrase[יִשְׁפְּט֣וּ ]
Genesis 31:53 clause[אֱלֹהֵ֨י אַבְרָהָ֜ם וֵֽאלֹהֵ֤י נָחֹור֙ ...] phrase[אֱלֹהֵ֨י אַבְרָהָ֜ם וֵֽאלֹהֵ֤י נָחֹור֙ ...] phrase[בֵינֵ֔ינוּ ]
1_Kings 18:40 clause[אִ֛ישׁ אַל־יִמָּלֵ֥ט מֵהֶ֖ם ] phrase[אִ֛ישׁ מֵהֶ֖ם ] phrase[יִמָּלֵ֥ט ]
1_Kings 18:40 clause[אִ֛ישׁ אַל־יִמָּלֵ֥ט מֵהֶ֖ם ] phrase[אִ֛ישׁ מֵהֶ֖ם ] phrase[אַל־]
2_Kings 20:14 clause

# <: (adjacent before) 
This relation holds when the left hand sides ends in a slot that lies before the first slot of the right hand side.
It serves to enforce an ordering between siblings of a parent.

# :> (adjacent after)
This relation holds when the left hand sides starts in a slot that lies after the last slot of the right hand side.

As an example: are there clauses with multiple clause atoms without a gap between the two?

In [54]:
query = '''
verse
    clause
        clause_atom
        <: clause_atom
'''
for r in S.search(query, limit=10): print(S.glean(r))
S.count(progress=1000, limit=-1)

  0.00s Counting results per 1000 up to  the end of the results ...
  0.76s Done: 0 results


Conclusion: there is always textual material between clause_atoms of the same clause.
If we lift the adjacency to sequentially before (`<<`) we do get results:

In [55]:
query = '''
verse
    clause
        clause_atom
        << clause_atom
'''
for r in S.search(query, limit=10): print(S.glean(r))
S.count(progress=1000, limit=-1)

Genesis 1:7 clause[וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙ ...] clause_atom[וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙ ] clause_atom[וּבֵ֣ין הַמַּ֔יִם ]
Genesis 1:11 clause[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] clause_atom[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ] clause_atom[עֵ֣ץ פְּרִ֞י ]
Genesis 1:11 clause[עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו עַל־...] clause_atom[עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו ] clause_atom[עַל־הָאָ֑רֶץ ]
Genesis 1:12 clause[וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא ...] clause_atom[וַתֹּוצֵ֨א הָאָ֜רֶץ דֶּ֠שֶׁא ...] clause_atom[וְעֵ֧ץ ]
Genesis 1:12 clause[עֹ֥שֶׂה פְּרִ֛י לְמִינֵ֑הוּ ] clause_atom[עֹ֥שֶׂה פְּרִ֛י ] clause_atom[לְמִינֵ֑הוּ ]
Genesis 1:21 clause[וַיִּבְרָ֣א אֱלֹהִ֔ים אֶת־הַ...] clause_atom[וַיִּבְרָ֣א אֱלֹהִ֔ים אֶת־הַ...] clause_atom[לְמִֽינֵהֶ֗ם ]
Genesis 1:29 clause[הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־...] clause_atom[הִנֵּה֩ נָתַ֨תִּי לָכֶ֜ם אֶת־כָּל־...] clause_atom[וְאֶת־כָּל־הָעֵ֛ץ ]
Genesis 1:30 clause[וּֽלְכָל־חַיַּ֣ת הָ֠...] clause_atom[וּֽלְכָל־חַיַּ֣ת הָ֠...] clause_atom[אֶת־כָּל־יֶ֥רֶ

# Nearness for := =: :: :> <:

The relations with `:` in their name always have a requirement somewhere that a slot of the
left hand node equals a slot of the right hand node, or that the two are adjacent.

All these relationships can be relaxed by a **nearness number**.
If you put a number *k* inside the relationship symbols, those restrictions will be relaxed to
*the one slot and the other slot should have a mutual distance of at most k*.

Here is an example.

First we look for clauses, with a phrase in it that starts at the
same slot as the clause.

In [56]:
S.study('''
chapter book=Genesis chapter=1
    clause
      =: phrase
''', silent=True)
S.count(progress=100, limit=-1)

  0.00s Counting results per 100 up to  the end of the results ...
   |     0.00s 100
  0.00s Done: 126 results


Now we add a bit of freedom, but not much: 0. Indeed, this is no extra
freedom, and it should give the same number of results.

In [57]:
S.study('''
chapter book=Genesis chapter=1
    clause
      =0: phrase
''', silent=True)
S.count(progress=100, limit=-1)

  0.00s Counting results per 100 up to  the end of the results ...
   |     0.00s 100
  0.00s Done: 126 results


Now we add real freedom: 1 and 2

In [58]:
S.study('''
chapter book=Genesis chapter=1
    clause
      =1: phrase
''', silent=True)
S.count(progress=100, limit=-1)

  0.00s Counting results per 100 up to  the end of the results ...
   |     0.00s 100
   |     0.00s 200
  0.01s Done: 236 results


In [59]:
S.study('''
chapter book=Genesis chapter=1
    clause
      =2: phrase
''', silent=True)
S.count(progress=100, limit=-1)

  0.00s Counting results per 100 up to  the end of the results ...
   |     0.00s 100
   |     0.00s 200
   |     0.01s 300
  0.01s Done: 315 results


Let us see some cases:

In [60]:
for r in S.fetch(limit=10): print(S.glean(r))

 clause[בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת ...] phrase[בָּרָ֣א ]
 clause[בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת ...] phrase[בְּרֵאשִׁ֖ית ]
 clause[וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ ...] phrase[וְ]
 clause[וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ ...] phrase[הָאָ֗רֶץ ]
 clause[וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום ] phrase[חֹ֖שֶׁךְ ]
 clause[וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום ] phrase[עַל־פְּנֵ֣י תְהֹ֑ום ]
 clause[וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום ] phrase[וְ]
 clause[וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־...] phrase[וְ]
 clause[וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־...] phrase[ר֣וּחַ אֱלֹהִ֔ים ]
 clause[וַיֹּ֥אמֶר אֱלֹהִ֖ים ] phrase[אֱלֹהִ֖ים ]


The first and second result show the same clause, with its first and second phrase respectively.

Note that we look for phrases that lie embedded in their clause.
So we do not get phrases of a preceding clause.

But if we want, we can get those as well.

In [61]:
S.study('''
chapter book=Genesis chapter=1
    c:clause
    p:phrase
    
    c =2: p
''', silent=True)
S.count(progress=100, limit=-1)

  0.00s Counting results per 100 up to  the end of the results ...
   |     0.00s 100
   |     0.00s 200
   |     0.01s 300
   |     0.01s 400
  0.01s Done: 485 results


We have more results now. Here is a closer look:

In [62]:
for r in S.search('''
verse book=Genesis chapter=1 verse=3
    c:clause
    p:phrase
    
    c =2: p
''', limit=100): print(S.glean(r))

S.count(progress=100, limit=-1)    

Genesis 1:3 clause[וַיֹּ֥אמֶר אֱלֹהִ֖ים ] phrase[אֱלֹהִ֖ים ]
Genesis 1:3 clause[וַיֹּ֥אמֶר אֱלֹהִ֖ים ] phrase[וַ]
Genesis 1:3 clause[וַיֹּ֥אמֶר אֱלֹהִ֖ים ] phrase[יֹּ֥אמֶר ]
Genesis 1:3 clause[יְהִ֣י אֹ֑ור ] phrase[אֱלֹהִ֖ים ]
Genesis 1:3 clause[יְהִ֣י אֹ֑ור ] phrase[יְהִ֣י ]
Genesis 1:3 clause[יְהִ֣י אֹ֑ור ] phrase[אֹ֑ור ]
Genesis 1:3 clause[יְהִ֣י אֹ֑ור ] phrase[וַֽ]
Genesis 1:3 clause[יְהִ֣י אֹ֑ור ] phrase[יֹּ֥אמֶר ]
Genesis 1:3 clause[וַֽיְהִי־אֹֽור׃ ] phrase[יְהִ֣י ]
Genesis 1:3 clause[וַֽיְהִי־אֹֽור׃ ] phrase[אֹ֑ור ]
Genesis 1:3 clause[וַֽיְהִי־אֹֽור׃ ] phrase[וַֽ]
Genesis 1:3 clause[וַֽיְהִי־אֹֽור׃ ] phrase[יְהִי־]
Genesis 1:3 clause[וַֽיְהִי־אֹֽור׃ ] phrase[אֹֽור׃ ]
  0.00s Counting results per 100 up to  the end of the results ...
  0.00s Done: 13 results


Here you see in result 4 a phrase of the previous clause in the result.

# Gaps

A question raised by Cody Kingham: **gaps**!

Search has no direct primitives to deal with gaps.
For example, the MQL query
```
SELECT ALL OBJECTS WHERE

[phrase FOCUS
    [word lex='L']
    [gap]
]
```
looks for a phrase with a gap in it
(i.e. one or more consecutive words between the start and the end of the phrase
that do not belong to the phrase).
The query then asks additionally for those gap-containing phrases that have a certain word in front of the gap.

Yet we can mimick this query in Search.

## Find the gap

In [63]:
query = '''
p:phrase
  =: wFirst:word
  wLast:word
  :=

wGap:word
wFirst < wGap
wGap < wLast
wGap || p
'''

In [64]:
S.study(query)

  0.00s Checking search template ...
  0.00s Setting up search space for 4 objects ...
  0.41s Constraining search space with 7 relations ...
  0.44s Setting up retrieval plan ...
  0.49s Ready to deliver results from 1532939 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results


In [65]:
S.showPlan(details=True)

Search with 4 objects and 7 relations
Results are instantiations of the following objects:
node  0-phrase                            (253187   choices)
node  1-word                              (426584   choices)
node  2-word                              (426584   choices)
node  3-word                              (426584   choices)
Instantiations are computed along the following relations:
node                      0-phrase        (253187   choices)
edge  0-phrase        :=  2-word          (     1.0 choices)
edge  2-word          ]]  0-phrase        (     1.0 choices)
edge  0-phrase        =:  1-word          (     1.0 choices)
edge  1-word          ]]  0-phrase        (     1.0 choices)
edge  2-word          >   3-word          (213292.0 choices)
edge  1-word          <   3-word          (213292.0 choices)
edge  3-word          ||  0-phrase        (227868.3 choices)
  2.75s The results are connected to the original search template as follows:
 0     
 1 R0  p:phrase
 2 R1    =: wFir

In [66]:
S.count(progress=2, limit=20)

  0.00s Counting results per 2 up to 20 ...
   |     8.27s 2
   |     8.27s 4
   |     8.27s 6
   |       15s 8
   |       17s 10
   |       17s 12
   |       46s 14
   |       46s 16
   |       46s 18
   |       46s 20
    46s Done: 20 results


It is not a fast query, to say the least.
Let's add an additional constraint, and see whether it goes faster.

In [23]:
query = '''
verse
    p:phrase
      =: wFirst:word
      wBefore:word lex=L
      wLast:word
      :=

wGap:word
wFirst < wGap
wGap < wLast
p || wGap
wBefore <: wGap
'''

In [24]:
S.study(query)

  0.00s Checking search template ...
  0.00s Setting up search space for 6 objects ...
  1.06s Constraining search space with 10 relations ...
  1.09s Setting up retrieval plan ...
  1.16s Ready to deliver results from 1576599 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results


In [25]:
S.showPlan(details=True)

Search with 6 objects and 10 relations
Results are instantiations of the following objects:
node  0-verse                             ( 23213   choices)
node  1-phrase                            (253187   choices)
node  2-word                              (426584   choices)
node  3-word                              ( 20447   choices)
node  4-word                              (426584   choices)
node  5-word                              (426584   choices)
Instantiations are computed along the following relations:
node                      3-word          ( 20447   choices)
edge  3-word          <:  5-word          (     1.0 choices)
edge  3-word          ]]  1-phrase        (     1.0 choices)
edge  5-word          ||  1-phrase        (227868.3 choices)
edge  1-phrase        ]]  0-verse         (     1.0 choices)
edge  1-phrase        :=  4-word          (     1.0 choices)
edge  4-word          ]]  1-phrase        (     1.0 choices)
edge  5-word          <   4-word          (213292.0 choi

In [26]:
S.count(progress=10)

  0.00s Counting results per 10 up to 1000 ...
   |     0.21s 10
  0.27s Done: 13 results


That is much quicker.
Let's see the results.

In [27]:
for r in S.fetch(): print(S.glean(r))

Leviticus 25:6 phrase[לָכֶם֙ לְךָ֖ וּלְעַבְדְּךָ֣ ...] לָכֶם֙  לָכֶם֙  תֹושָׁ֣בְךָ֔  לְ
Genesis 17:7 phrase[לְךָ֙ וּֽלְזַרְעֲךָ֖ אַחֲרֶֽיךָ׃ ] לְךָ֙  לְךָ֙  אַחֲרֶֽיךָ׃  לֵֽ
Deuteronomy 26:11 phrase[לְךָ֛ וּלְבֵיתֶ֑ךָ ] לְךָ֛  לְךָ֛  בֵיתֶ֑ךָ  יְהוָ֥ה 
Exodus 30:21 phrase[לָהֶ֧ם לֹ֥ו וּלְזַרְעֹ֖ו ] לָהֶ֧ם  לָהֶ֧ם  זַרְעֹ֖ו  חָק־
Genesis 28:4 phrase[לְךָ֙ לְךָ֖ וּלְזַרְעֲךָ֣ ...] לְךָ֙  לְךָ֙  אִתָּ֑ךְ  אֶת־
2_Kings 25:24 phrase[לָהֶ֤ם וּלְאַנְשֵׁיהֶ֔ם ] לָהֶ֤ם  לָהֶ֤ם  אַנְשֵׁיהֶ֔ם  גְּדַלְיָ֨הוּ֙ 
Daniel 9:8 phrase[לָ֚נוּ לִמְלָכֵ֥ינוּ לְשָׂרֵ֖ינוּ ...] לָ֚נוּ  לָ֚נוּ  אֲבֹתֵ֑ינוּ  בֹּ֣שֶׁת 
Genesis 31:16 phrase[לָ֥נוּ וּלְבָנֵ֑ינוּ ] לָ֥נוּ  לָ֥נוּ  בָנֵ֑ינוּ  ה֖וּא 
Numbers 20:15 phrase[לָ֛נוּ וְלַאֲבֹתֵֽינוּ׃ ] לָ֛נוּ  לָ֛נוּ  אֲבֹתֵֽינוּ׃  מִצְרַ֖יִם 
Numbers 32:33 phrase[לָהֶ֣ם׀ לִבְנֵי־גָד֩ וְ...] לָהֶ֣ם׀  לָהֶ֣ם׀  יֹוסֵ֗ף  מֹשֶׁ֡ה 
1_Samuel 25:31 phrase[לְךָ֡ לַאדֹנִ֗י ] לְךָ֡  לְךָ֡  אדֹנִ֗י  לְ
Jeremiah 40:9 phrase[לָהֶ֜ם וּלְאַנְשֵׁיהֶ֣ם ] לָהֶ֜ם  לָהֶ֜ם  אַנְשֵׁיהֶ֣ם  גְּדַ

In [28]:
B.show(S.fetch(), start=0)


##### Result 0



##### Result 1



##### Result 2



##### Result 3



##### Result 4


...


##### Result 8



##### Result 9



##### Result 10



##### Result 11



##### Result 12
